In [ ]:
pip install --upgrade transformers datasets wandb huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.9
    Uninstalling wandb-0.19.9:
      Successfully uninstalled wandb-0.19.9


In [ ]:
!pip install --upgrade transformers



In [ ]:
# -*- coding: utf-8 -*-
from datasets import load_dataset
import transformers
import wandb
from huggingface_hub import login, create_repo, upload_folder
import os

# ----------------------------------------
# 1. Weights & Biases Login
# ----------------------------------------
wandb.login()

# ----------------------------------------
# 2. HF Tokenizer und Modell laden
# ----------------------------------------
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")

# ----------------------------------------
# 3. Datensatz laden
# ----------------------------------------
dataset = load_dataset("karpathy/tiny_shakespeare")

# ----------------------------------------
# 4. Tokenisierung
# ----------------------------------------
def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=False)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# ----------------------------------------
# 5. Texte in Blöcke gruppieren
# ----------------------------------------
block_size = 512

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = (len(concatenated["input_ids"]) // block_size) * block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True, batch_size=1000)

# ----------------------------------------
# 6. Data Collator
# ----------------------------------------
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ----------------------------------------
# 7. TrainingArguments + Trainer
# ----------------------------------------
training_args = TrainingArguments(
    output_dir="./gpt2-shakespeare",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    report_to="wandb",  # Wichtig!
    run_name="gpt2-shakespeare-run",
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["train"].select(range(100)),  # optionales Eval-Set
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ----------------------------------------
# 8. Training starten
# ----------------------------------------
trainer.train()

# ----------------------------------------
# 9. Speichern & Upload zu Hugging Face
# ----------------------------------------
model.push_to_hub("gpt2-shakespeare-finetuned")
tokenizer.push_to_hub("gpt2-shakespeare-finetuned")

# Optional: Upload mit huggingface_hub
# login(token="your-hf-token")
# upload_folder("gpt2-shakespeare", repo_id="username/gpt2-shakespeare-finetuned")

# ----------------------------------------
# 10. WandB Run abschließen
# ----------------------------------------
wandb.finish()


<ipython-input-16-e853814bb0a6>:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
295,3.627500


Step,Training Loss
295,3.627500
590,3.356500
885,3.254300


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

train/epoch,▁▅██
train/global_step,▁▅██
train/grad_norm,▁█▆
train/learning_rate,█▅▁
train/loss,█▃▁
total_flos,461703020544000.0
train/epoch,3
train/global_step,885
train/grad_norm,7.84287
train/learning_rate,0.0
train/loss,3.2543
